<a href="https://colab.research.google.com/github/kirinotkiri/Capstone-Project-Peluang/blob/main/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/translated_dataset_hve.csv')

# Display the first few rows
df.head()

,DATE,DESC
0,24/11/2018,KABEL
1,24/11/2018,KABEL
2,20/12/2018,BEARING GEAR BROKEN
3,03/01/2019,FILTER SOLAR EDC
4,15/01/2019,JALUR PUMP SPREADER


In [2]:
# Convert the 'DATE START' column to datetime
df['DATE'] = pd.to_datetime(df['DATE'], format='%d/%m/%Y')

# Strip any leading/trailing whitespace from the 'STRIPPED' column
df['DESC'] = df['DESC'].str.strip()

# Split the 'STRIPPED' column into individual words
df['WORDS'] = df['DESC'].str.split()

# Display the cleaned data
df.head()


,DATE,DESC,WORDS
0,2018-11-24,KABEL,[KABEL]
1,2018-11-24,KABEL,[KABEL]
2,2018-12-20,BEARING GEAR BROKEN,"[BEARING, GEAR, BROKEN]"
3,2019-01-03,FILTER SOLAR EDC,"[FILTER, SOLAR, EDC]"
4,2019-01-15,JALUR PUMP SPREADER,"[JALUR, PUMP, SPREADER]"


In [9]:
word_pairs = [
    ("SEAL", "TROMOL"),
    ("SAMBUNG", "CHAIN"),
    ("SAMBUNG", "KABEL"),
    ("BAUT", "PENGIKAT"),
    ("BAUT", "STEERING"),
    ("BAUT", "WHEEL"),
    ("BAUT", "KOPEL"),
    ("PERBAIKI", "LEAK"),
    ("HYDRAULIC", "LEAK"),
    ("REORING", "VALVE"),
    ("REORING", "SELANG"),
    ("SUPPLY", "SENSOR"),
    ("PRESSURE", "OIL"),
    ("BROKEN", "SELANG"),
    ("BROKEN", "REMOVE"),
    ("STEERING", "LEAK"),
    ("BUKA", "WHEEL"),
    ("BUKA", "SELANG"),
    ("BUKA", "TROMOL"),
    ("AIR", "RADIATOR"),
    ("WHEEL", "KEMPES"),
    ("WHEEL", "SPARE"),
    ("WHEEL", "WHEEL"),
    ("WHEEL", "DEPAN"),
    ("WHEEL", "PCS"),
    ("WHEEL", "LEAK"),
    ("HOUSE", "FUSE"),
    ("ORING", "SELANG"),
    ("ADJUST", "CHAIN"),
    ("ADJUST", "SENSOR"),
    ("TAMBAH", "OIL"),
    ("CLEAN", "KABEL"),
    ("BONGKAR", "WHEEL"),
    ("REPAIR", "LEAK"),
    ("GEAR", "CHAIN"),
    ("VALVE", "LEAK"),
    ("BRAKE", "BLONG"),
    ("BRAKE", "LEAK"),
    ("FILTER", "SOLAR"),
    ("BOOM", "LIFT"),
    ("MESIN", "HIDUP"),
    ("CHARGING", "VALVE"),
    ("ISI", "AIR"),
    ("ISI", "OIL"),
    ("SELANG", "HYDRAULIC"),
    ("SELANG", "BRAKE"),
    ("SELANG", "SELANG"),
    ("SELANG", "LOCK"),
    ("SELANG", "PUMP"),
    ("SELANG", "LS"),
    ("SELANG", "BALIKAN"),
    ("SELANG", "RADIATOR"),
    ("SELANG", "LEAK"),
    ("LINK", "ARM"),
    ("LOCK", "PEN"),
    ("LOCK", "MATI"),
    ("LOCK", "UNLOCK"),
    ("LOCK", "LEAK"),
    ("LOCK", "SENSOR"),
    ("STARTER", "DINAMO"),
    ("PERBAIKAN", "KABEL"),
    ("CHAIN", "SPIDER"),
    ("CHAIN", "PUTUS"),
    ("CHAIN", "KENDOR"),
    ("HIGH", "PRESSURE"),
    ("KABEL", "SAMBUNG"),
    ("KABEL", "SUPPLY"),
    ("KABEL", "HARNES"),
    ("KABEL", "CHAIN"),
    ("KABEL", "PUTUS"),
    ("KABEL", "KABEL"),
    ("KABEL", "FEEDBACK"),
    ("KABEL", "SENSOR"),
    ("JALUR", "BAKAR"),
    ("PIN", "LOCK"),
    ("FEEDBACK", "SENSOR"),
    ("PEN", "PORTUDING"),
    ("PEN", "SPIDER"),
    ("PEN", "LINK"),
    ("OIL", "HYDRAULIC"),
    ("OIL", "TRANSMISSION"),
    ("OIL", "BRAKE"),
    ("OIL", "MESIN"),
    ("OIL", "DIFFERENTIAL"),
    ("PUMP", "HYDRAULIC"),
    ("PUMP", "OIL"),
    ("PUMP", "LEAK"),
    ("KENCANGKAN", "BAUT"),
    ("KENCANGKAN", "CHAIN"),
    ("ERROR", "KABEL"),
    ("ERROR", "SENSOR"),
    ("MATI", "KABEL"),
    ("DINAMO", "CURRENT"),
    ("DINAMO", "STARTER"),
    ("REMOVE", "BAUT"),
    ("REMOVE", "WHEEL"),
    ("REMOVE", "SELANG"),
    ("REMOVE", "CHAIN"),
    ("REMOVE", "PEN"),
    ("REMOVE", "DINAMO"),
    ("REMOVE", "HOSE"),
    ("BEARING", "GEAR"),
    ("DUDUKAN", "SENSOR"),
    ("HOSE", "LS"),
    ("HOSE", "LEAK"),
    ("FUSE", "PUTUS"),
    ("MAJU", "MUNDUR"),
    ("RADIATOR", "LEAK"),
    ("LEAK", "BUKA"),
    ("LEAK", "AIR"),
    ("LEAK", "ORING"),
    ("LEAK", "SELANG"),
    ("LEAK", "KARIS"),
    ("LEAK", "OIL"),
    ("LEAK", "REMOVE"),
    ("SELENOID", "VALVE"),
    ("KENDOR", "KENCANGKAN"),
    ("LAMP", "LOCK"),
    ("SENSOR", "PRESSURE"),
    ("SENSOR", "ALIGMENT"),
    ("SENSOR", "LOCK"),
    ("SENSOR", "KABEL"),
    ("SENSOR", "SENSOR")
]
# Create a new DataFrame to store pair occurrences
pair_occurrences = pd.DataFrame(columns=['Pair', 'DATE'])

# Iterate through word pairs and find occurrences in the original DataFrame (df)
for pair in word_pairs:
    word1, word2 = pair
    # Find rows where both words in the pair occur
    mask = df['DESC'].str.contains(word1) & df['DESC'].str.contains(word2)
    pair_df = df[mask][['DATE']].copy()
    pair_df['Pair'] = '_'.join(pair)
    pair_occurrences = pd.concat([pair_occurrences, pair_df])

# Calculate average time between occurrences and count for each pair
pair_diffs = {}
pair_counts = {}
for pair, group in pair_occurrences.groupby('Pair'):
    group = group.sort_values(by='DATE')
    date_diffs = group['DATE'].diff().dropna().dt.days
    pair_diffs[pair] = date_diffs
    pair_counts[pair] = group.shape[0]  # Count the occurrences

avg_days_between_pairs = {pair: diffs.mean() for pair, diffs in pair_diffs.items()}

# Create a DataFrame for results
result_df_pairs = pd.DataFrame({
    'Pair': list(avg_days_between_pairs.keys()),
    'Average Days Between': list(avg_days_between_pairs.values()),
    'Count': list(pair_counts.values())  # Add the count column
})

# Display the results
print(result_df_pairs)

<ipython-input-9-99f6130af99d>:136: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pair_occurrences = pd.concat([pair_occurrences, pair_df])


              Pair  Average Days Between  Count
0     ADJUST_CHAIN            116.375000      9
1    ADJUST_SENSOR            182.125000      9
2     AIR_RADIATOR             60.960000     26
3       BAUT_KOPEL            355.750000      5
4    BAUT_PENGIKAT            248.333333      7
..             ...                   ...    ...
118   WHEEL_KEMPES            108.166667     13
119     WHEEL_LEAK             72.142857     22
120      WHEEL_PCS             73.000000     17
121    WHEEL_SPARE             96.555556     10
122    WHEEL_WHEEL             22.105882     86

[123 rows x 3 columns]


In [10]:
# prompt: code to print the dataframe with merged words into a csv

# Save the results to a CSV file
pair_occurrences.to_csv('merged_words.csv', index=False)
print("CSV file 'merged_words.csv' created successfully.")


CSV file 'merged_words.csv' created successfully.


In [8]:
# prompt: code to print the result into a csv

result_df_pairs.to_csv('pair_occurrences.csv', index=False)


Prediction test

In [17]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import numpy as np
from datetime import datetime

# Load the dataset
# Assuming your dataset is in a file named 'dataset.csv'
df = pd.read_csv('merged_words.csv')

# Preprocess the data
# Combine the pair values (order doesn't matter) by sorting the words in each pair
df['Pair'] = df['Pair'].apply(lambda x: '_'.join(sorted(x.split('_'))))

# Convert the Date column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d')

# Encode the 'Pair' labels
label_encoder = LabelEncoder()
df['Pair_Encoded'] = label_encoder.fit_transform(df['Pair'])

# Create features and target variable
# Using Pair_Encoded as a feature and predicting the next number of days until the next occurrence
df['Next_Date'] = df.groupby('Pair')['DATE'].shift(-1)
df['Days_Until_Next'] = (df['Next_Date'] - df['DATE']).dt.days

# Drop rows where Next_Date is NaN (i.e., the last occurrence of each pair)
df = df.dropna(subset=['Next_Date'])

# Split the data into training and testing sets
X = df[['Pair_Encoded', 'DATE']]
X['DATE'] = X['DATE'].apply(lambda x: x.toordinal())
y = df['Days_Until_Next']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict the number of days until the next occurrence
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Prediction function
def predict_next_occurrence(pair, last_date):
    pair_encoded = label_encoder.transform(['_'.join(sorted(pair.split('_')))])
    last_date_ordinal = datetime.strptime(last_date, '%d/%m/%Y').toordinal()
    prediction = model.predict([[pair_encoded[0], last_date_ordinal]])
    next_occurrence_date = datetime.fromordinal(last_date_ordinal + int(prediction[0]))
    return next_occurrence_date.strftime('%d/%m/%Y')

# Example usage
pair = "SENSOR_KABEL"
last_date = "14/07/2019"
next_date = predict_next_occurrence(pair, last_date)
print(f'The predicted next occurrence of {pair} after {last_date} is on {next_date}')


<ipython-input-17-628172b18b78>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['DATE'] = X['DATE'].apply(lambda x: x.toordinal())


Mean Absolute Error: 53.52699520306141
The predicted next occurrence of SENSOR_KABEL after 14/07/2019 is on 23/08/2019


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
